In [1]:
pip install --upgrade websockets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Sistema Propuesto

Este sistema implementa un monitor de trading automatizado que se conecta a Finnhub mediante WebSocket para recibir datos de mercado en tiempo real. El sistema analiza tendencias utilizando el indicador ADX (Average Directional Index) y puede generar alertas basadas en condiciones específicas del mercado.

In [3]:
# Importación de bibliotecas necesarias
import pandas as pd                    # Para manipulación y análisis de datos
from datetime import datetime, timedelta # Para manejo de fechas y tiempos
import websockets                      # Para conexiones websocket
import asyncio                         # Para programación asíncrona
import json                           # Para manejo de datos JSON
import numpy as np                    # Para cálculos numéricos
import nest_asyncio                   # Para permitir loops asyncio anidados
import time                           # Para funciones relacionadas con tiempo
from IPython.display import clear_output # Para limpiar salida en notebooks
import threading                      # Para manejo de hilos
from concurrent.futures import ThreadPoolExecutor # Para pool de hilos
import pytz                          # Para manejo de zonas horarias
from ta.trend import ADXIndicator    # Para el indicador técnico ADX
import logging                       # Para registro de eventos

# Habilitar loops anidados para Jupyter
nest_asyncio.apply()

class ConnectionStatus:
    """
    Clase que define los estados posibles de la conexión al websocket.
    Utiliza constantes en español para mejor comprensión en el contexto local.
    """
    DISCONNECTED = "Desconectado"     # Estado cuando no hay conexión
    CONNECTING = "Conectando"         # Estado durante el intento de conexión
    CONNECTED = "Conectado"           # Estado cuando la conexión está establecida
    ERROR = "Error"                   # Estado cuando ocurre un error
    RECONNECTING = "Reconectando"     # Estado durante un intento de reconexión

class TradingConfig:
    """
    Clase principal de configuración para el sistema de trading.
    Maneja los parámetros de configuración y constantes necesarias para la operación.
    """
    def __init__(self,
                symbol,                    # Símbolo del instrumento financiero
                adx_threshold=25,          # Umbral para considerar tendencia fuerte
                di_threshold=20,           # Umbral para señales de cruce DI
                alert_cooldown=300,        # Tiempo mínimo entre alertas (en segundos)
                data_window=300,           # Ventana de tiempo para datos históricos
                update_interval=2,         # Frecuencia de actualización de cálculos
                api_key=None):             # Clave API para autenticación
        
        # Configuración básica
        self.symbol = symbol.upper()        # Convertir símbolo a mayúsculas
        self.adx_threshold = adx_threshold  # Umbral ADX para detectar tendencia fuerte
        self.di_threshold = di_threshold    # Umbral para detectar cruces significativos
        self.alert_cooldown = alert_cooldown # Evita múltiples alertas en poco tiempo
        self.data_window = data_window      # Cantidad de datos históricos a mantener
        self.update_interval = update_interval # Frecuencia de actualizaciones
        
        # Configuración de la API
        self.ws_url = "wss://ws.finnhub.io"  # URL del websocket de Finnhub
        self.api_key = api_key or "cukll41r01qo08i8jju0cukll41r01qo08i8jjug" # API key con fallback
        
        # Parámetros de reconexión
        self.max_retries = 3                # Número máximo de intentos de reconexión
        self.reconnect_delay = 5            # Tiempo entre intentos de reconexión

In [4]:
!python -m pip install ta



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import time
from datetime import datetime

class AlertSystem:
    def __init__(self):
        """
        Constructor de la clase AlertSystem.
        Inicializa las variables para rastrear la última alerta y la última señal generada.
        """
        self.last_alert = None  # Marca de tiempo de la última alerta enviada
        self.last_signal = None  # Última señal de trading generada (BUY, SELL o NEUTRAL)
        
    def check_signals(self, data, config):
        """
        Analiza los datos de entrada y genera señales de trading, incluyendo estado neutral.
        
        Parámetros:
        - data (dict): Diccionario con los datos del mercado, que debe incluir:
            'ADX': Índice de Movimiento Direccional Medio (Average Directional Index).
            'DI+': Indicador Direccional Positivo (+DI).
            'DI-': Indicador Direccional Negativo (-DI).
            'timestamp': Marca de tiempo del dato en segundos.
            'close': Precio de cierre del activo en el momento de análisis.
        - config (objeto): Configuración con los siguientes atributos:
            - adx_threshold: Umbral mínimo de ADX para considerar una tendencia fuerte.
            - di_threshold: Umbral mínimo de DI+ o DI- para confirmar una tendencia.
            - alert_cooldown: Tiempo mínimo (en segundos) entre alertas repetitivas.

        Retorna:
        - dict: Mensaje de alerta con el estado actual del mercado si se genera una señal nueva.
        - None: Si no se genera una nueva alerta (para evitar repetición de señales).
        """
        current_time = time.time()  # Obtiene la marca de tiempo actual
        
        # Extrae los indicadores técnicos desde el diccionario de datos
        adx = data['ADX']
        di_plus = data['DI+']
        di_minus = data['DI-']
        
        # Determinar el estado del mercado con base en ADX y DI+ / DI-
        if adx > config.adx_threshold:  # ADX por encima del umbral → tendencia fuerte
            if di_plus > di_minus and di_plus > config.di_threshold:
                signal_type = 'BUY'  # Señal de compra
                market_state = 'Tendencia alcista fuerte'
            elif di_minus > di_plus and di_minus > config.di_threshold:
                signal_type = 'SELL'  # Señal de venta
                market_state = 'Tendencia bajista fuerte'
            else:
                signal_type = 'NEUTRAL'  # Tendencia fuerte sin dirección clara
                market_state = 'Tendencia fuerte sin dirección clara'
        else:
            signal_type = 'NEUTRAL'  # ADX por debajo del umbral → tendencia débil o mercado en rango
            if adx < 20:
                market_state = 'Mercado sin tendencia clara - Posible rango lateral'
            else:
                market_state = 'Tendencia débil - Esperar confirmación'

        # Evitar alertas repetitivas para BUY/SELL
        if (signal_type in ['BUY', 'SELL'] and 
            self.last_signal == signal_type and 
            self.last_alert is not None and 
            current_time - self.last_alert < config.alert_cooldown):
            return None  # Se evita enviar una alerta repetitiva en un corto período de tiempo

        # Actualizar la última alerta y la última señal registrada
        self.last_alert = current_time
        self.last_signal = signal_type
        
        # Generar y retornar el mensaje de alerta
        return self._generate_alert(data, signal_type, market_state)
        
    def _generate_alert(self, data, signal_type, market_state):
        """
        Genera un mensaje de alerta con los detalles del mercado.
        
        Parámetros:
        - data (dict): Diccionario con los datos de mercado.
        - signal_type (str): Tipo de señal generada ('BUY', 'SELL' o 'NEUTRAL').
        - market_state (str): Descripción del estado del mercado.

        Retorna:
        - dict: Mensaje de alerta con la información relevante del mercado.
        """
        return {
            'type': signal_type,  # Tipo de señal generada
            'state': market_state,  # Estado descriptivo del mercado
            'timestamp': datetime.fromtimestamp(data['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),  # Fecha y hora formateada
            'price': data['close'],  # Precio de cierre del activo
            'adx': data['ADX'],  # Valor de ADX
            'di_plus': data['DI+'],  # Valor de DI+
            'di_minus': data['DI-']  # Valor de DI-
        }

In [6]:
class StockDataStream:
    def __init__(self, config=None):
        """
        Inicializa la clase StockDataStream, que gestiona la recepción de datos en tiempo real a través de WebSockets.

        Parámetros:
        - config (TradingConfig, opcional): Configuración del sistema de trading. Si no se proporciona, se usa una configuración predeterminada.
        """
        self.config = config or TradingConfig()  # Carga la configuración o usa la predeterminada
        self.data_buffer = []  # Buffer para almacenar los datos de precios en tiempo real
        self.last_update = None  # Última actualización de datos
        self.running = True  # Estado de ejecución del stream
        self.connection_status = ConnectionStatus.DISCONNECTED  # Estado de conexión inicial
        self.messages_received = 0  # Contador de mensajes recibidos
        self.alert_system = AlertSystem()  # Sistema de alertas para señales de compra y venta
        self.websocket = None  # Conexión WebSocket (inicialmente nula)
        self.task = None  # Tarea asíncrona para manejar el WebSocket
        self.loop = None  # Event loop de asyncio
        self.connection_retries = 0  # Contador de intentos de reconexión

        # Configuración del sistema de logging para registrar eventos
        self.setup_logger()

    def setup_logger(self):
        """
        Configura el sistema de logging para registrar eventos del stream de datos en un archivo de log.
        
        - El nombre del archivo de log sigue el formato 'trading_alerts_<symbol>_<YYYYMMDD>.log'.
        - Se usa un nivel de logging INFO para capturar eventos importantes.
        - Se evita agregar múltiples manejadores al logger si ya existen.
        """
        
        # Obtiene la fecha actual en formato YYYYMMDD para incluirla en el nombre del archivo de log
        fecha_actual = datetime.now().strftime('%Y%m%d')
        
        # Define el nombre del archivo de log con el símbolo de la acción y la fecha actual
        log_filename = f'trading_alerts_{self.config.symbol}_{fecha_actual}.log'

        # Crea un logger específico para el símbolo de la acción
        self.logger = logging.getLogger(f'trading_{self.config.symbol}')
        
        # Establece el nivel de logging en INFO para registrar eventos relevantes
        self.logger.setLevel(logging.INFO)

        # Verifica si el logger ya tiene manejadores para evitar duplicados
        if not self.logger.handlers:
            # Crea un manejador de archivo para guardar los logs
            file_handler = logging.FileHandler(log_filename)
            
            # Define el formato de los logs: timestamp | nivel de log | mensaje
            file_formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
            
            # Asigna el formato al manejador
            file_handler.setFormatter(file_formatter)
            
            # Agrega el manejador de archivo al logger
            self.logger.addHandler(file_handler)

    def log_alert(self, alert):
        """
        Registra una alerta en el sistema de logging.

        Parámetros:
        alert (dict): Un diccionario que contiene los detalles de la alerta.
            - 'type' (str): Tipo de alerta ('BUY', 'SELL' o estado neutral).
            - 'price' (float): Precio de la acción en el momento de la alerta.
            - 'adx' (float): Indicador ADX en el momento de la alerta.
            - 'di_plus' (float): Valor del indicador DI+.
            - 'di_minus' (float): Valor del indicador DI-.
            - 'state' (str): Estado de la tendencia.

        Funcionamiento:
        - Si la alerta es de tipo 'BUY' o 'SELL', se registra con nivel INFO.
        - Si la alerta es neutral, se registra con nivel DEBUG.
        """

        # Verifica si la alerta es de compra ('BUY') o venta ('SELL')
        if alert['type'] in ['BUY', 'SELL']:
            # Formatea el mensaje de la alerta con los indicadores técnicos relevantes
            mensaje = (
                f"ALERTA DE {alert['type']} | "
                f"Símbolo: {self.config.symbol} | "
                f"Precio: ${alert['price']:.2f} | "
                f"ADX: {alert['adx']:.2f} | "
                f"DI+: {alert['di_plus']:.2f} | "
                f"DI-: {alert['di_minus']:.2f} | "
                f"Estado: {alert['state']}"
            )
            # Registra la alerta con nivel INFO en el log
            self.logger.info(mensaje)
        
        else:
            # Si la alerta no es de compra ni venta, se considera un estado neutral
            mensaje = (
                f"ESTADO NEUTRAL | "
                f"Símbolo: {self.config.symbol} | "
                f"Precio: ${alert['price']:.2f} | "
                f"Estado: {alert['state']}"
            )
            # Registra la alerta con nivel DEBUG, ya que no requiere acción inmediata
            self.logger.debug(mensaje)

    def _get_signal_emoji(self, signal_type):
        """
        Retorna el emoji correspondiente a un tipo de señal de trading.

        Parámetros:
        signal_type (str): Tipo de señal, puede ser 'BUY', 'SELL' o 'NEUTRAL'.

        Retorna:
        str: Emoji que representa visualmente la señal.

        Mapeo de señales:
        - 'BUY' (Compra) → 🟢
        - 'SELL' (Venta) → 🔴
        - 'NEUTRAL' (Sin tendencia definida) → ⚪
        - Cualquier otro valor → ❓ (desconocido)
        """

        # Diccionario que asocia cada tipo de señal con su emoji correspondiente
        emojis = {
            'BUY': '🟢',     # Señal de compra
            'SELL': '🔴',    # Señal de venta
            'NEUTRAL': '⚪'  # Señal neutral
        }
        
        # Retorna el emoji correspondiente o '❓' si el tipo de señal no está definido
        return emojis.get(signal_type, '❓')

    def calculate_adx(self, df, period=14):
        """
        Calcula el indicador ADX (Average Directional Index) y sus componentes DI+ y DI-.

        Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'high', 'low' y 'close' de los precios del activo.
        period (int): Período de cálculo del ADX (por defecto 14).

        Retorna:
        tuple(pd.Series, pd.Series, pd.Series): 
            - ADX (Serie de valores de tendencia)
            - DI+ (Serie de valores positivos de la dirección del índice)
            - DI- (Serie de valores negativos de la dirección del índice)
        """

        # Verifica que haya suficientes datos para calcular el ADX
        if len(df) < period:
            return pd.Series(np.nan, index=df.index)

        # Crea una instancia del indicador ADX utilizando la librería ta (technical analysis)
        adx_indicator = ADXIndicator(
            high=df['high'],   # Precios máximos
            low=df['low'],     # Precios mínimos
            close=df['close'], # Precios de cierre
            window=period      # Período del indicador
        )

        # Retorna el ADX y sus componentes DI+ y DI-
        return adx_indicator.adx(), adx_indicator.adx_pos(), adx_indicator.adx_neg()

    async def print_summary(self, data):
        """
        Genera y muestra un resumen en la consola con información clave del mercado.

        Parámetros:
        data (dict): Diccionario con los datos más recientes del mercado, incluyendo precios, indicadores y señales.

        Funcionalidad:
        - Limpia la consola antes de mostrar el resumen.
        - Presenta los datos del activo, incluyendo el precio actual y los indicadores ADX, DI+ y DI-.
        - Evalúa si hay una señal de trading (BUY, SELL o NEUTRAL).
        - Muestra la señal correspondiente con emojis para una mejor visualización.
        - Registra las alertas en el log de la aplicación.
        - Imprime información sobre la conexión y la cantidad de mensajes recibidos.
        - Maneja posibles errores durante la ejecución e informa en el log.
        """

        try:
            # Limpia la salida de la consola para mostrar siempre información actualizada
            clear_output(wait=True)
            
            # Encabezado del resumen de mercado
            print("\n" + "="*50)
            print(f"RESUMEN DE MERCADO - {self.config.symbol}")
            print("="*50)

            # Muestra la fecha y hora del último dato recibido
            print(f"\nFecha y Hora: {datetime.fromtimestamp(data['timestamp']).strftime('%Y-%m-%d %H:%M:%S')}")

            # Muestra los precios e indicadores técnicos
            print("\nPRECIOS E INDICADORES:")
            print(f"Precio actual: ${data['close']:.2f}")
            print(f"ADX: {data['ADX']:.2f}")
            print(f"DI+: {data['DI+']:.2f}")
            print(f"DI-: {data['DI-']:.2f}")

            # Analiza si hay una señal de trading
            alert = self.alert_system.check_signals(data, self.config)

            print("\nANÁLISIS DE MERCADO:")
            print("-" * 30)

            if alert:
                # Obtiene el emoji correspondiente a la señal (ejemplo: 🟢 para BUY)
                emoji = self._get_signal_emoji(alert['type'])
                print(f"SEÑAL ACTUAL: {emoji} {alert['type']}")
                print(f"Estado: {alert['state']}")

                # Si la señal es de compra o venta, muestra una alerta destacada
                if alert['type'] in ['BUY', 'SELL']:
                    print("\n" + "!"*50)
                    print(f"🚨 ALERTA DE TRADING - {alert['type']}")
                    print(f"Precio: ${alert['price']:.2f}")
                    print(f"ADX: {alert['adx']:.2f}")
                    print(f"DI+: {alert['di_plus']:.2f}")
                    print(f"DI-: {alert['di_minus']:.2f}")
                    print("!"*50)

                # Registra la alerta en el log
                self.log_alert(alert)

            # Muestra el estado de la conexión y el número de mensajes recibidos
            print(f"\nESTADO DE CONEXIÓN: {self.connection_status}")
            print(f"Mensajes recibidos: {self.messages_received}")

            # Indica la ubicación del archivo de log donde se almacenan las alertas
            print(f"Log: trading_alerts_{self.config.symbol}_{datetime.now().strftime('%Y%m%d')}.log")
            print("="*50)

        except Exception as e:
            # Captura errores y los muestra en la consola, además de registrarlos en el log
            print(f"Error en print_summary: {e}")
            self.logger.error(f"Error en print_summary: {e}")

    async def handle_trade_message(self, message):
        """
        Maneja los mensajes recibidos en tiempo real desde el WebSocket de trading.

        Parámetros:
        message (str): Mensaje en formato JSON recibido desde el WebSocket.

        Funcionalidad:
        - Incrementa el contador de mensajes recibidos.
        - Procesa los mensajes de tipo 'ping' y responde con 'pong' para mantener la conexión activa.
        - Extrae los datos de las transacciones si el mensaje es de tipo 'trade'.
        - Agrega los datos al buffer, asegurando que solo se mantengan los datos dentro de la ventana de tiempo configurada.
        - Calcula los indicadores ADX, DI+ y DI- si hay suficientes datos.
        - Llama a `print_summary` para actualizar el resumen si ha pasado el tiempo mínimo de actualización.
        - Maneja posibles errores y los registra en el log.
        """

        try:
            # Incrementa el contador de mensajes recibidos
            self.messages_received += 1

            # Decodifica el mensaje JSON recibido
            data = json.loads(message)

            # Si el mensaje es un 'ping', responde con 'pong' para mantener la conexión activa
            if data.get('type') == 'ping':
                if self.websocket:
                    await self.websocket.send(json.dumps({'type': 'pong'}))
                return  # Sale de la función ya que no hay más procesamiento necesario
            
            # Procesa solo los mensajes de tipo 'trade' que contengan datos
            if data['type'] == 'trade' and 'data' in data:
                trades = data['data']
                
                for trade in trades:
                    timestamp = trade['t'] / 1000  # Convierte el timestamp de milisegundos a segundos
                    price = float(trade['p'])  # Extrae el precio de la transacción
                    
                    # Agrega los datos de la transacción al buffer, asignando valores de precio a cierre, máximo y mínimo
                    self.data_buffer.append({
                        'timestamp': timestamp,
                        'close': price,
                        'high': price,
                        'low': price,
                    })
                
                # Filtra el buffer para mantener solo los datos dentro del intervalo configurado
                current_time = time.time()
                self.data_buffer = [d for d in self.data_buffer 
                                    if current_time - d['timestamp'] <= self.config.data_window]

                # Calcula los indicadores ADX, DI+ y DI- solo si hay suficientes datos en el buffer
                if len(self.data_buffer) > 14:
                    df = pd.DataFrame(self.data_buffer)  # Convierte el buffer en un DataFrame
                    adx, di_pos, di_neg = self.calculate_adx(df)  # Calcula los indicadores
                    
                    # Agrega los indicadores calculados al DataFrame
                    df['ADX'] = adx
                    df['DI+'] = di_pos
                    df['DI-'] = di_neg
                    
                    # Verifica si ha pasado el tiempo mínimo de actualización antes de imprimir el resumen
                    if (self.last_update is None or 
                        current_time - self.last_update >= self.config.update_interval):
                        self.last_update = current_time  # Actualiza el tiempo de la última actualización
                        await self.print_summary(df.iloc[-1])  # Llama a print_summary con el último dato disponible

        except Exception as e:
            # Maneja cualquier error en la ejecución y lo registra en el log
            print(f"Error procesando mensaje: {e}")
            self.logger.error(f"Error procesando mensaje: {e}")

    async def connect_websocket(self):
        """
        Establece y mantiene la conexión WebSocket con el servidor de datos financieros.

        Funcionalidad:
        - Intenta conectar al WebSocket hasta alcanzar el número máximo de reintentos.
        - Maneja errores de conexión y reintenta en caso de fallos.
        - Se suscribe a los datos de la acción configurada.
        - Procesa los mensajes recibidos y los envía a `handle_trade_message`.
        - Controla el estado de la conexión e imprime información relevante.

        Parámetros:
        - No recibe parámetros directamente, pero usa `self.config` para obtener la configuración.

        Estado de conexión:
        - CONNECTING: Intentando conectar al WebSocket.
        - CONNECTED: Conexión establecida exitosamente.
        - ERROR: Ocurrió un error y se intentará reconectar.
        """

        while self.running:  # Mientras el stream esté activo, intenta mantener la conexión
            try:
                # Si se supera el número máximo de reintentos, detiene la conexión
                if self.connection_retries >= self.config.max_retries:
                    error_msg = f"\n❌ Se superó el número máximo de intentos ({self.config.max_retries})"
                    print(error_msg)
                    self.logger.error(error_msg)
                    self.running = False  # Finaliza el bucle
                    break

                # Indica que se está intentando conectar
                self.connection_status = ConnectionStatus.CONNECTING
                print(f"\n🔄 Intento de conexión {self.connection_retries + 1}/{self.config.max_retries}")

                # Intenta establecer la conexión con el servidor WebSocket
                async with websockets.connect(f"{self.config.ws_url}?token={self.config.api_key}") as websocket:
                    self.websocket = websocket  # Guarda la conexión activa
                    self.connection_status = ConnectionStatus.CONNECTED  # Marca la conexión como establecida
                    
                    # Registra e imprime la conexión exitosa
                    self.logger.info(f"Conexión WebSocket establecida para {self.config.symbol}")
                    print(f"\n🟢 Conexión WebSocket establecida para {self.config.symbol}")

                    # Envía mensaje de suscripción para recibir datos de la acción especificada
                    subscribe_message = {
                        "type": "subscribe",
                        "symbol": self.config.symbol
                    }
                    await websocket.send(json.dumps(subscribe_message))
                    print("✅ Suscripción enviada")

                    # Loop para recibir y manejar mensajes en tiempo real mientras el WebSocket esté activo
                    while self.running:
                        try:
                            message = await websocket.recv()  # Recibe un mensaje
                            await self.handle_trade_message(message)  # Procesa el mensaje recibido
                        except websockets.exceptions.ConnectionClosed:
                            # Si la conexión se cierra inesperadamente, registra y avisa
                            print("\n🔴 Conexión cerrada")
                            self.logger.warning("Conexión WebSocket cerrada")
                            break  # Sale del loop interno para intentar reconectar

            except Exception as e:
                # Captura cualquier error de conexión y actualiza el estado
                self.connection_status = ConnectionStatus.ERROR
                self.connection_retries += 1  # Aumenta el contador de intentos fallidos
                
                error_msg = f"\n❌ Error de conexión: {e}"
                print(error_msg)
                self.logger.error(error_msg)

                if self.running:
                    # Espera antes de reintentar la conexión
                    print(f"Reintentando en {self.config.reconnect_delay} segundos...")
                    await asyncio.sleep(self.config.reconnect_delay)

    def is_market_open(self):
        """
        Determina si el mercado de valores de EE.UU. (NYSE/NASDAQ) está abierto en tiempo real.

        Retorna:
        - (bool) True si el mercado está abierto, False si está cerrado.
        - (str) Mensaje indicando el estado actual del mercado.
        - (datetime) Próxima apertura del mercado si está cerrado.

        Consideraciones:
        - El horario del mercado es de 9:30 AM a 4:00 PM (hora del este, US/Eastern).
        - Los fines de semana (sábado y domingo), el mercado está cerrado.
        - Se maneja la apertura del siguiente día hábil si el mercado está cerrado.
        """
        
        # Definir la zona horaria del mercado (Hora del Este, EE.UU.)
        est = pytz.timezone('US/Eastern')
        now = datetime.now(est)  # Obtener la hora actual en la zona horaria de NY
        
        # Verificar si hoy es fin de semana (sábado=5, domingo=6)
        if now.weekday() in [5, 6]:  # Si es sábado o domingo
            next_open = now
            while next_open.weekday() in [5, 6]:  # Buscar el siguiente día hábil (lunes)
                next_open += timedelta(days=1)
            next_open = next_open.replace(hour=9, minute=30, second=0, microsecond=0)  # Fijar la próxima apertura
            return False, "Mercado cerrado (fin de semana)", next_open  # Retorna estado y próxima apertura

        # Definir horarios de apertura y cierre del mercado
        market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
        market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)

        # Si la hora actual es antes de la apertura, indicar pre-market
        if now < market_open:
            return False, "Mercado cerrado (pre-market)", market_open  # Próxima apertura a las 9:30 AM

        # Si la hora actual es después del cierre, indicar post-market y calcular la próxima apertura
        if now >= market_close:
            next_day = now + timedelta(days=1)  # Ir al siguiente día
            while next_day.weekday() in [5, 6]:  # Saltar fines de semana
                next_day += timedelta(days=1)
            next_open = next_day.replace(hour=9, minute=30, second=0, microsecond=0)  # Fijar apertura del próximo día hábil
            return False, "Mercado cerrado (post-market)", next_open

        # Si ninguna de las condiciones anteriores se cumple, el mercado está abierto
        return True, "Mercado abierto", market_close  # Indica el estado abierto y la próxima hora de cierre

    def start(self):
        """
        Inicia el stream de datos verificando primero si el mercado está abierto.

        - Si el mercado está cerrado, muestra un mensaje de alerta con la próxima apertura y detiene el programa.
        - Si el mercado está abierto, inicia el loop de eventos asíncrono en un hilo separado para manejar la conexión WebSocket.

        Retorna:
        - (bool) True si el stream inicia correctamente.
        - (bool) False si el mercado está cerrado y no se inicia el stream.
        """
        
        # Verificar si el mercado está abierto
        market_open, status, next_open = self.is_market_open()
        
        # Si el mercado está cerrado, mostrar advertencia y detener el proceso
        if not market_open:
            print("\n" + "=" * 50)
            print("⚠️ ALERTA DE MERCADO CERRADO")
            print("=" * 50)
            print(f"\nEstado: {status}")
            
            # Obtener la zona horaria local y convertir la próxima apertura a la hora local del usuario
            local_tz = datetime.now().astimezone().tzinfo
            next_open_local = next_open.astimezone(local_tz)
            
            print(f"\nPróxima apertura del mercado:")
            print(f"Fecha: {next_open_local.strftime('%Y-%m-%d')}")
            print(f"Hora: {next_open_local.strftime('%H:%M:%S')} (hora local)")
            print("\nEl programa se detendrá.")
            
            # Registrar advertencia en el log
            self.logger.warning(f"Intento de inicio con mercado cerrado: {status}")
            return False  # No se inicia el stream

        # Configurar la ejecución en un hilo separado
        self.running = True
        self.executor = ThreadPoolExecutor(max_workers=1)
        self.loop = asyncio.new_event_loop()
        
        def run_async_loop():
            """Ejecuta el loop asíncrono en un hilo separado para manejar WebSockets."""
            asyncio.set_event_loop(self.loop)
            self.task = self.loop.create_task(self.connect_websocket())
            try:
                self.loop.run_until_complete(self.task)
            except asyncio.CancelledError:
                pass  # Manejar cancelaciones seguras del loop

        # Crear y lanzar un hilo para ejecutar el loop de eventos asíncrono
        self.thread = threading.Thread(target=run_async_loop)
        self.thread.start()
        
        # Registrar en logs y mostrar mensaje de inicio
        self.logger.info("Stream iniciado")
        print("Stream iniciado. Para detener, usa stream.stop()")
        
        return True  # Stream iniciado exitosamente

    def stop(self):
        """ 
        Método para detener el stream de datos de manera segura.

        - Cambia la bandera `self.running` a False para indicar que el proceso debe detenerse.
        - Cancela la tarea asíncrona `self.task` si está en ejecución.
        - Cierra la conexión WebSocket si está activa.
        - Espera a que el hilo de ejecución principal termine de forma ordenada.
        - Apaga el `ThreadPoolExecutor` sin esperar a que finalicen todas las tareas.
        - Registra en logs y muestra un mensaje en consola indicando que el stream ha sido detenido.
        """
        
        print("\n🛑 Deteniendo el stream de datos...")
        self.running = False  # Señal para detener la ejecución

        # Si la tarea asíncrona `self.task` está en ejecución, se cancela de forma segura
        if hasattr(self, 'task') and self.task:
            self.loop.call_soon_threadsafe(self.task.cancel)

        # Si la conexión WebSocket está activa, se cierra de forma segura
        if hasattr(self, 'websocket') and self.websocket:
            self.loop.call_soon_threadsafe(lambda: asyncio.ensure_future(self.websocket.close()))

        # Espera a que el hilo de ejecución termine, con un tiempo límite de 5 segundos
        if hasattr(self, 'thread'):
            self.thread.join(timeout=5)

        # Cierra el `ThreadPoolExecutor`, deteniendo todas las tareas en segundo plano
        if hasattr(self, 'executor'):
            self.executor.shutdown(wait=False)

        # Registrar en el log que el stream ha sido detenido
        self.logger.info("Stream detenido")
        print("Stream detenido exitosamente")


In [7]:
def create_stock_stream(symbol, adx_threshold=25, di_threshold=20, api_key=None):
    """
    Crea y retorna una instancia del stream con configuración personalizada.
    
    Parámetros:
    - symbol (str): Símbolo de la acción a monitorear (ej: 'AAPL', 'MSFT', 'GOOGL').
    - adx_threshold (float): Umbral mínimo del ADX para considerar una tendencia fuerte (default: 25).
    - di_threshold (float): Umbral mínimo de los indicadores DI+ y DI- para confirmar la dirección de la tendencia (default: 20).
    - api_key (str): Clave de la API de Finnhub para la conexión con WebSockets (opcional).
    
    Retorna:
    - StockDataStream: Instancia del stream configurado con los parámetros especificados.
    """
    
    # Se crea una instancia de TradingConfig con los parámetros proporcionados.
    config = TradingConfig(
        symbol=symbol,  # Símbolo de la acción
        adx_threshold=adx_threshold,  # Umbral para identificar tendencias fuertes
        di_threshold=di_threshold,  # Umbral para confirmar dirección de la tendencia
        api_key=api_key  # Clave de la API para la conexión con Finnhub
    )
    
    # Mensajes informativos sobre la configuración del sistema
    print("\nIniciando sistema de trading en tiempo real...")
    print("\nConfiguración:")
    print(f"Símbolo: {config.symbol}")  # Muestra el símbolo de la acción
    print(f"Umbral ADX: {config.adx_threshold}")  # Muestra el umbral de ADX configurado
    print(f"Umbral DI: {config.di_threshold}")  # Muestra el umbral de DI configurado
    print(f"Tiempo entre alertas: {config.alert_cooldown}s")  # Muestra el tiempo mínimo entre alertas
    
    # Se crea una instancia del flujo de datos bursátiles con la configuración establecida
    stream = StockDataStream(config)
    
    # Retorna la instancia del stream para su ejecución y monitoreo
    return stream

In [8]:
# Crear una instancia del stream para AAPL (Apple)
stream = create_stock_stream(
    symbol="NVDA",         # Símbolo de la acción
    adx_threshold=25,      # Umbral para considerar una tendencia fuerte
    di_threshold=20        # Umbral para confirmar dirección de tendencia
)

# Iniciar el stream
stream.start()

# El stream comenzará a mostrar actualizaciones en tiempo real
# Para detenerlo cuando quieras, ejecuta:
#stream.stop()


Iniciando sistema de trading en tiempo real...

Configuración:
Símbolo: NVDA
Umbral ADX: 25
Umbral DI: 20
Tiempo entre alertas: 300s

⚠️ ALERTA DE MERCADO CERRADO

Estado: Mercado cerrado (fin de semana)

Próxima apertura del mercado:
Fecha: 2025-03-03
Hora: 09:30:00 (hora local)

El programa se detendrá.


False

# Prueba del sistema

In [9]:
import pandas as pd
from datetime import datetime, timedelta
import websockets
import asyncio
import json
import numpy as np
import nest_asyncio
import time
from IPython.display import clear_output
import threading
from concurrent.futures import ThreadPoolExecutor
import pytz
from ta.trend import ADXIndicator
import logging

# Enable nested loops for Jupyter
nest_asyncio.apply()

class ConnectionStatus:
    DISCONNECTED = "Desconectado"
    CONNECTING = "Conectando"
    CONNECTED = "Conectado"
    ERROR = "Error"
    RECONNECTING = "Reconectando"

class TradingConfig:
    def __init__(self, 
                 symbol,                    # Símbolo obligatorio
                 adx_threshold=25,          # Umbral ADX para tendencia fuerte
                 di_threshold=20,           # Umbral para cruces DI
                 alert_cooldown=300,        # Tiempo entre alertas (segundos)
                 data_window=300,           # Ventana de datos (segundos)
                 update_interval=2,         # Intervalo de actualización (segundos)
                 api_key=None):             # API key opcional
        
        self.symbol = symbol.upper()        # Convertir a mayúsculas
        self.adx_threshold = adx_threshold
        self.di_threshold = di_threshold
        self.alert_cooldown = alert_cooldown
        self.data_window = data_window
        self.update_interval = update_interval
        
        # Configuración de API
        self.ws_url = "wss://ws.finnhub.io"
        self.api_key = api_key or "cukll41r01qo08i8jju0cukll41r01qo08i8jjug"
        self.max_retries = 3
        self.reconnect_delay = 5

In [10]:
class AlertSystem:
    def __init__(self):
        self.last_alert = None
        self.last_signal = None
        
    def check_signals(self, data, config):
        """
        Analiza los datos y genera señales de trading incluyendo estado neutral
        Retorna: (dict) mensaje de alerta con el estado actual del mercado
        """
        current_time = time.time()
        adx = data['ADX']
        di_plus = data['DI+']
        di_minus = data['DI-']
        
        # Determinar el estado del mercado
        if adx > config.adx_threshold:
            if di_plus > di_minus and di_plus > config.di_threshold:
                signal_type = 'BUY'
                market_state = 'Tendencia alcista fuerte'
            elif di_minus > di_plus and di_minus > config.di_threshold:
                signal_type = 'SELL'
                market_state = 'Tendencia bajista fuerte'
            else:
                signal_type = 'NEUTRAL'
                market_state = 'Tendencia fuerte sin dirección clara'
        else:
            signal_type = 'NEUTRAL'
            if adx < 20:
                market_state = 'Mercado sin tendencia clara - Posible rango lateral'
            else:
                market_state = 'Tendencia débil - Esperar confirmación'

        # Evitar alertas repetitivas para BUY/SELL
        if (signal_type in ['BUY', 'SELL'] and 
            self.last_signal == signal_type and 
            self.last_alert is not None and 
            current_time - self.last_alert < config.alert_cooldown):
            return None

        self.last_alert = current_time
        self.last_signal = signal_type
        
        return self._generate_alert(data, signal_type, market_state)
        
    def _generate_alert(self, data, signal_type, market_state):
        return {
            'type': signal_type,
            'state': market_state,
            'timestamp': datetime.fromtimestamp(data['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
            'price': data['close'],
            'adx': data['ADX'],
            'di_plus': data['DI+'],
            'di_minus': data['DI-']
        }

In [11]:
class StockDataStream:
    def __init__(self, config=None):
        self.config = config or TradingConfig()
        self.data_buffer = []
        self.last_update = None
        self.running = True
        self.connection_status = ConnectionStatus.DISCONNECTED
        self.messages_received = 0
        self.alert_system = AlertSystem()
        self.websocket = None
        self.task = None
        self.loop = None
        self.connection_retries = 0
        # Variables para simulación
        self.base_price = 150.0
        self.trend = 0
        self.volatility = 0.02
        # Configuración del logger
        self.setup_logger()

    def setup_logger(self):
        """Configura el sistema de logging"""
        fecha_actual = datetime.now().strftime('%Y%m%d')
        log_filename = f'trading_alerts_{self.config.symbol}_{fecha_actual}.log'
        
        self.logger = logging.getLogger(f'trading_{self.config.symbol}')
        self.logger.setLevel(logging.INFO)
        
        if not self.logger.handlers:
            file_handler = logging.FileHandler(log_filename)
            file_formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
            file_handler.setFormatter(file_formatter)
            self.logger.addHandler(file_handler)

    def log_alert(self, alert):
        """Registra una alerta en el log"""
        if alert['type'] in ['BUY', 'SELL']:
            mensaje = (
                f"ALERTA DE {alert['type']} | "
                f"Símbolo: {self.config.symbol} | "
                f"Precio: ${alert['price']:.2f} | "
                f"ADX: {alert['adx']:.2f} | "
                f"DI+: {alert['di_plus']:.2f} | "
                f"DI-: {alert['di_minus']:.2f} | "
                f"Estado: {alert['state']}"
            )
            self.logger.info(mensaje)
        else:
            mensaje = (
                f"ESTADO NEUTRAL | "
                f"Símbolo: {self.config.symbol} | "
                f"Precio: ${alert['price']:.2f} | "
                f"Estado: {alert['state']}"
            )
            self.logger.debug(mensaje)

    def _get_signal_emoji(self, signal_type):
        """Retorna el emoji apropiado para cada tipo de señal"""
        emojis = {
            'BUY': '🟢',
            'SELL': '🔴',
            'NEUTRAL': '⚪'
        }
        return emojis.get(signal_type, '❓')

    def calculate_adx(self, df, period=14):
        if len(df) < period:
            return pd.Series(np.nan, index=df.index)
        
        adx_indicator = ADXIndicator(
            high=df['high'],
            low=df['low'],
            close=df['close'],
            window=period
        )
        
        return adx_indicator.adx(), adx_indicator.adx_pos(), adx_indicator.adx_neg()

    def _simulate_price(self):
        """Simula movimientos de precio más realistas"""
        # Cambiar tendencia ocasionalmente
        if random.random() < 0.05:  # 5% de probabilidad de cambio de tendencia
            self.trend = random.uniform(-0.1, 0.1)
        
        # Simular movimiento de precio
        price_change = (
            self.trend +  # Componente de tendencia
            random.normalvariate(0, self.volatility)  # Ruido aleatorio
        )
        
        self.base_price = max(self.base_price * (1 + price_change), 1.0)
        current_price = self.base_price
        
        # Simular high y low
        volatility_range = self.volatility * 0.5
        high = current_price * (1 + random.uniform(0, volatility_range))
        low = current_price * (1 - random.uniform(0, volatility_range))
        
        # Asegurar que high > low
        high, low = max(high, low), min(high, low)
        
        return {
            'timestamp': time.time(),
            'close': current_price,
            'high': high,
            'low': low
        }

    async def print_summary(self, data):
        try:
            clear_output(wait=True)
            print("\n" + "="*50)
            print(f"RESUMEN DE MERCADO - {self.config.symbol} (SIMULACIÓN)")
            print("="*50)
            
            print(f"\nFecha y Hora: {datetime.fromtimestamp(data['timestamp']).strftime('%Y-%m-%d %H:%M:%S')}")
            
            print("\nPRECIOS E INDICADORES:")
            print(f"Precio actual: ${data['close']:.2f}")
            print(f"ADX: {data['ADX']:.2f}")
            print(f"DI+: {data['DI+']:.2f}")
            print(f"DI-: {data['DI-']:.2f}")
            
            alert = self.alert_system.check_signals(data, self.config)
            
            print("\nANÁLISIS DE MERCADO:")
            print("-" * 30)
            
            if alert:
                emoji = self._get_signal_emoji(alert['type'])
                print(f"SEÑAL ACTUAL: {emoji} {alert['type']}")
                print(f"Estado: {alert['state']}")
                
                if alert['type'] in ['BUY', 'SELL']:
                    print("\n" + "!"*50)
                    print(f"🚨 ALERTA DE TRADING - {alert['type']}")
                    print(f"Precio: ${alert['price']:.2f}")
                    print(f"ADX: {alert['adx']:.2f}")
                    print(f"DI+: {alert['di_plus']:.2f}")
                    print(f"DI-: {alert['di_minus']:.2f}")
                    print("!"*50)
                
                self.log_alert(alert)
            
            print(f"\nESTADO: Simulación en ejecución")
            print(f"Actualizaciones: {self.messages_received}")
            print(f"Log: trading_alerts_{self.config.symbol}_{datetime.now().strftime('%Y%m%d')}.log")
            print("="*50)
        except Exception as e:
            print(f"Error en print_summary: {e}")
            self.logger.error(f"Error en print_summary: {e}")

    async def simulate_trading(self):
        """Simula datos de trading"""
        self.connection_status = ConnectionStatus.CONNECTED
        print("Inicializando datos históricos...")
        
        # Inicializar el buffer con datos históricos
        for _ in range(20):
            trade_data = self._simulate_price()
            self.data_buffer.append(trade_data)
            await asyncio.sleep(0.1)
        
        print("Comenzando simulación en tiempo real...")
        
        while self.running:
            try:
                self.messages_received += 1
                
                trade_data = self._simulate_price()
                self.data_buffer.append(trade_data)
                
                current_time = time.time()
                self.data_buffer = [d for d in self.data_buffer 
                                  if current_time - d['timestamp'] <= self.config.data_window]
                
                if len(self.data_buffer) > 14:
                    df = pd.DataFrame(self.data_buffer)
                    adx, di_pos, di_neg = self.calculate_adx(df)
                    
                    latest_data = {
                        'timestamp': trade_data['timestamp'],
                        'close': trade_data['close'],
                        'high': trade_data['high'],
                        'low': trade_data['low'],
                        'ADX': float(adx.iloc[-1] if not pd.isna(adx.iloc[-1]) else 0),
                        'DI+': float(di_pos.iloc[-1] if not pd.isna(di_pos.iloc[-1]) else 0),
                        'DI-': float(di_neg.iloc[-1] if not pd.isna(di_neg.iloc[-1]) else 0)
                    }
                    
                    await self.print_summary(latest_data)
                
                await asyncio.sleep(1)
                
            except Exception as e:
                print(f"Error en simulación: {e}")
                self.logger.error(f"Error en simulación: {e}")
                await asyncio.sleep(1)

    def is_market_open(self):
        return True, "Simulación activa", datetime.now()

    def start(self):
        """Inicia la simulación"""
        self.running = True
        self.executor = ThreadPoolExecutor(max_workers=1)
        self.loop = asyncio.new_event_loop()
        
        def run_simulation():
            asyncio.set_event_loop(self.loop)
            self.task = self.loop.create_task(self.simulate_trading())
            try:
                self.loop.run_until_complete(self.task)
            except asyncio.CancelledError:
                pass
            
        self.thread = threading.Thread(target=run_simulation)
        self.thread.start()
        print("Simulación iniciada. Para detener, usa stream.stop()")
        return True

    def stop(self):
        """Detiene la simulación"""
        print("\n🛑 Deteniendo simulación...")
        self.running = False
        
        if hasattr(self, 'task') and self.task:
            self.loop.call_soon_threadsafe(self.task.cancel)
        
        if hasattr(self, 'thread'):
            self.thread.join(timeout=5)
        
        if hasattr(self, 'executor'):
            self.executor.shutdown(wait=False)
        
        self.logger.info("Simulación detenida")
        print("Simulación detenida exitosamente")

In [12]:
def create_stock_stream(symbol, adx_threshold=25, di_threshold=20, api_key=None):
    """
    Crea y retorna una instancia del stream con configuración personalizada
    
    Parámetros:
    symbol (str): Símbolo de la acción (ej: 'AAPL', 'MSFT', 'GOOGL')
    adx_threshold (float): Umbral para considerar tendencia fuerte (default: 25)
    di_threshold (float): Umbral para confirmar dirección de tendencia (default: 20)
    api_key (str): API key de Finnhub (opcional)
    """
    config = TradingConfig(
        symbol=symbol,
        adx_threshold=adx_threshold,
        di_threshold=di_threshold,
        api_key=api_key
    )
    
    print("\nIniciando sistema de trading en tiempo real...")
    print("\nConfiguración:")
    print(f"Símbolo: {config.symbol}")
    print(f"Umbral ADX: {config.adx_threshold}")
    print(f"Umbral DI: {config.di_threshold}")
    print(f"Tiempo entre alertas: {config.alert_cooldown}s")
    
    stream = StockDataStream(config)
    return stream

In [17]:
import random

# Crear el stream de prueba
test_stream = create_stock_stream(
    symbol="NVDA",
    adx_threshold=25,
    di_threshold=20
)

# Iniciar la simulación
test_stream.start()

# Para detener cuando quieras:
# test_stream.stop()


RESUMEN DE MERCADO - NVDA (SIMULACIÓN)

Fecha y Hora: 2025-03-01 13:26:12

PRECIOS E INDICADORES:
Precio actual: $1.01
ADX: 96.37
DI+: 4.70
DI-: 78.80

ANÁLISIS DE MERCADO:
------------------------------

ESTADO: Simulación en ejecución
Actualizaciones: 105
Log: trading_alerts_NVDA_20250301.log


In [18]:
test_stream.stop()


🛑 Deteniendo simulación...
Simulación detenida exitosamente
